In [86]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package wordnet to /home/danish/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/danish/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [144]:
df = pd.read_csv("dataset/data.txt", delimiter=',',encoding = "utf-8")

In [145]:
df['text'] = [entry.lower() for entry in df['text']]
df['text']= [word_tokenize(entry) for entry in df['text']]

In [ ]:
Corpus  = df
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['text']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    Corpus.loc[index,'text'] = str(Final_words)
Corpus.head()

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text'],Corpus['label'],test_size=0.2)

In [134]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

# SVM

In [135]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("Accuracy -> ",accuracy_score( Test_Y,predictions_SVM)*100)
print("F1 Score -> ",f1_score(Test_Y,predictions_SVM, average="macro"))
print("Precision -> ",precision_score(Test_Y, predictions_SVM, average="macro"))
print("Recall -> ",recall_score(Test_Y,predictions_SVM, average="macro"))   

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

# Random Forest

In [104]:
RFC = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0)
RFC.fit(Train_X_Tfidf,Train_Y)
predictions_RFC = RFC.predict(Test_X_Tfidf)
print("Accuracy -> ",accuracy_score( Test_Y,predictions_RFC)*100)
print("F1 Score -> ",f1_score(Test_Y,predictions_RFC, average="macro"))
print("Precision -> ",precision_score(Test_Y, predictions_RFC, average="macro"))
print("Recall -> ",recall_score(Test_Y,predictions_RFC, average="macro"))  

Accuracy ->  73.07692307692307
F1 Score ->  0.44304584304584305
Precision ->  0.864570737605804
Recall ->  0.5109170305676856


# Logistic Regression

In [106]:
LR = LogisticRegression(random_state=0)
LR.fit(Train_X_Tfidf,Train_Y)
predictions_LR = LR.predict(Test_X_Tfidf)
print("Accuracy -> ",accuracy_score(Test_Y,predictions_LR)*100)
print("F1 Score -> ",f1_score(Test_Y,predictions_LR, average="macro"))
print("Precision -> ",precision_score(Test_Y, predictions_LR, average="macro"))
print("Recall -> ",recall_score(Test_Y,predictions_LR, average="macro"))  

Accuracy ->  87.13942307692307
F1 Score ->  0.8223977815682637
Precision ->  0.8699211259452224
Recall ->  0.7961683576296104


In [33]:
import json
encoding = 'utf-8'
count = 0
f = open("negative_data.txt", "w")
f.write("text,label\n")
with open('final_15K_dataset.json') as json_file:
    data = json.load(json_file)
    for text,label in zip(data['text'],data['label']):
        if(label==0):
            f.write(text.replace('\n',' ').encode("ascii", "ignore").decode(encoding)+","+str(label)+"\n")
            count+=1
        if(count==3360):
            break

f.close()
json_file.close()

print(count)

3360
